# ROOT

## Setting
Import the Python module for ROOT, and specify the path to the BNet file containing the Boolean model logic to be analyzed, as well as the directory where the results will be saved.

In [1]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

#Add the path to the folder containing the modules of the ROOT framework to sys.path.
import sys, os

path_of_this_notebook = os.getcwd()
path_of_ROOT_framework = os.path.dirname(os.path.dirname(path_of_this_notebook))
# 'path_of_ROOT_framework' should direct the 'ROOT framework' folder. if not, please edit it manually.

sys.path.append(path_of_ROOT_framework)

In [2]:
#import python modules of ROOT

from Model_read_using_pyboolnet import read_pyboolnet_file
from iATG_module import iATG

In [3]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

# the path to the .bnet file containing the logic of the model to the variables.

model_address = os.path.join(path_of_this_notebook, 'HIV_latency_model.bnet')

## Determine transition-determining input configurations to analyze
Specify the input configurations to be analyzed.<br>
The input configuration before the irreversible process occurs is referred to as the basal input configuration ($IC_{basal}$), and the one that induces the irreversible process is referred to as the transition input configuration ($IC_{trans}$).<br><br>
For each node included in the input configuration, create a dictionary where the node is the key and the Boolean value it has in the basal input configuration is the value. Store this dictionary in a variable named `IC_basal`.<br>
Similarly, create a dictionary for the transition input configuration and store it in the variable `IC_transition`.

If mutations are to be incorporated into the model, the corresponding mutation information should be provided.<br>
Mutations can be represented as the fixation of specific nodes in the network model to either the ON (1) or OFF (0) state.<br>
This information should be entered into the `mutated_node_state_map` dictionary, where the key is the name of the mutated node and the value is the fixed Boolean state assigned to that node.

In [4]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

IC_basal = {"TNFalpha":0, "HMTs_input":0}
IC_transition = {"TNFalpha":1, "HMTs_input":0}
mutated_node_state_map = {}

## Construct an iATG
After creating the iATG and dynamics objects, perform the computation to construct the iATG by setting the appropriate parameter values.

In [5]:
dynamics_of_model = read_pyboolnet_file(model_address)
iATG_of_model = iATG(dynamics_of_model, IC_basal, IC_transition, mutated_node_state_map)

### Set the parameter values
- Set the parameter values required to compute the model’s iATG:
    - `use_all_initials`: Set this to either `True` or `False`.
        - If `True`, the attractor landscape for each input configuration is computed using all possible initial states of the Boolean model.
        - If `False`, only a subset of possible initial states is used to estimate the attractor landscape.
            - The calculation method of the approximated attractor landscape is explained in the Supplementary Materials
    - `waiting_number`: A positive integer.
        - Used _only_ when `use_all_initials` is `False`.
        - The larger this value is, the more initial states are used for attractor landscape estimation, resulting in higher estimation accuracy.
    - `difference_threshold`: A positive real number less than 1.
        - Used _only_ when `use_all_initials` is `False`.
        - The smaller this value is, the more initial states are used for attractor landscape estimation, improving accuracy.
    - `verbose`: Set this to either True or False.
        - When `use_all_initials` is `False`, setting `verbose` to `True` will display how many initial states are being explored during attractor landscape estimation.

In [6]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

use_all_initials=True
waiting_number = 10000
difference_threshold = 0.0001
verbose=False

using these parameters, iATG of the model is constructed

In [7]:
iATG_of_model.calculate_attractor_landscapes_for_each_IC(use_all_initials,
                                                        waiting_number, 
                                                        difference_threshold,
                                                        verbose)
iATG_of_model.get_attractor_transitions_induced_by_IC_change_and_calculate_TPs()

Calculating attractor landscapes using all initial states.

Calculating attractor landscape on input configuration {'TNFalpha': 0, 'HMTs_input': 0} using all initial states.
0.0% of all initial states are detected.
used time:  0.0
20.000076293945312% of all initial states are detected.
used time:  419.29858326911926
40.000152587890625% of all initial states are detected.
used time:  1259.3955459594727
60.00022888183594% of all initial states are detected.
used time:  2495.675763130188
80.00030517578125% of all initial states are detected.
used time:  4451.050616979599
time for Attractor landscape for IC {'TNFalpha': 0, 'HMTs_input': 0}:  6541.751084089279

Calculating attractor landscape on input configuration {'TNFalpha': 1, 'HMTs_input': 0} using all initial states.
0.0% of all initial states are detected.
used time:  0.0
20.000076293945312% of all initial states are detected.
used time:  423.9611086845398
40.000152587890625% of all initial states are detected.
used time:  1158.99572

## Check the constructed iATG and select appropriate ITPs.
After inspecting the constructed iATG, examine the irreversible transition paths (ITPs) present in it.<br>By analyzing the phenotypes of the attractors that compose each ITP, select the ITPs that correspond to the observed irreversibile phenotypic change (OIPC).

### constructed iATG
The cell below displays the constructed iATG.<br>
Attractors included in the attractor landscape under the basal input configuration (`IC_basal`) are labeled with attractor codes in the form of `('basal', i)`, where `i` is an arbitrary index number.<br>
Attractors under the transition input configuration (`IC_transition`) are labeled as `('transition', i)`, where `i` is also an arbitrary index.<br><br>
An arrow between attractor codes (e.g., `('basal', 0) → ('transition', 1)`) indicates that if the model state remains at the attractor on the left (in this case, `('basal', 0)`), a transition to the attractor on the right (in this case, `('transition', 1)`) can occur upon a change in input configuration.

In [8]:
print("iATG constructed. \nthe edges in the iATG are as follows:")
for attractor_transition in iATG_of_model.attractor_transitions_induced_by_IC_change:
    print("{}->{}".format(attractor_transition[0],attractor_transition[1]))

iATG constructed. 
the edges in the iATG are as follows:
('basal', 0)->('transition', 0)
('basal', 1)->('transition', 0)
('basal', 1)->('transition', 1)
('basal', 2)->('transition', 0)
('basal', 3)->('transition', 0)
('basal', 4)->('transition', 5)
('transition', 0)->('basal', 3)
('transition', 1)->('basal', 1)
('transition', 2)->('basal', 2)
('transition', 3)->('basal', 1)
('transition', 4)->('basal', 1)
('transition', 5)->('basal', 4)


### Phenotype check of ITPs
Using the constructed iATG, irreversible transition paths (ITPs) present in the iATG can be identified.<br>
The ITPs that exist in this iATG are listed in the output of the following cell.



In [9]:
print("ITPs in the constructed iATG are as follows:")
ITP_index_map = {}
ITP_index = 0
iATG_of_model.find_iCAs_and_calculate_iCA_sizes()
for iCA in iATG_of_model.iCAs:
    iCA.search_ITPs()
    for ITP_of_model in iCA.ITPs:
        ITP_index_map[ITP_index] = ITP_of_model
        print("ITP index {}: {}".format(ITP_index, ITP_of_model))
        ITP_index += 1

ITPs in the constructed iATG are as follows:
ITP index 0: ITP from ('basal', 0) to ('transition', 0) to ('basal', 3)
ITP index 1: ITP from ('basal', 1) to ('transition', 0) to ('basal', 3)
ITP index 2: ITP from ('basal', 2) to ('transition', 0) to ('basal', 3)


Among the given ITPs, those corresponding to the observed irreversible phenotypic change (OIPC) should be selected for analysis.<br>
To do this, the average state information of the attractors that make up each ITP is utilized in this established model.

In [10]:
import pandas as pd

ITP_info_for_index = []
for index_of_ITP, ITP_of_model in ITP_index_map.items():
    ITP_info_for_index.append((index_of_ITP, str(ITP_of_model.attr_basal_irrev)))
    ITP_info_for_index.append((index_of_ITP, str(ITP_of_model.attr_transition)))
    ITP_info_for_index.append((index_of_ITP, str(ITP_of_model.attr_basal_rev)))


index_of_df = pd.MultiIndex.from_tuples(ITP_info_for_index, names=["ITP index", "attractor code"])

is_point_attractor = []
node_names_input_first = list(IC_basal.keys())
for node_name in dynamics_of_model.get_node_names():
    if node_name not in node_names_input_first:
        node_names_input_first.append(node_name)
node_averagestates_map = {node_name:[] for node_name in node_names_input_first}
for index_of_ITP, attractor_code in ITP_info_for_index:
    attractor_code = eval(attractor_code)#str to tuple
    attractor_object = iATG_of_model.get_attractor_using_attr_tuple_form(attractor_code)
    average_state = attractor_object.get_average_state()
    is_point_attractor.append(attractor_object.is_point_attractor())
    for node_name, list_of_average_states in node_averagestates_map.items():
        average_state_of_node = average_state.get(node_name, None)
        list_of_average_states.append(average_state_of_node)

df_for_ITP_info = pd.DataFrame({"is point attractor":is_point_attractor, **node_averagestates_map}, 
                               index=index_of_df)

The `df_for_ITP_info` calculated in the cell above contains the average state information of attractors included in each ITP.<br>
Among the rows with the same ITP index value, 
- the first row shows the average state of the attractor corresponding to the ITP's first attractor, 
- the second row corresponds to the second attractor of the ITP, 
- and the third row corresponds to the third (final) attractor of the ITP.

<br>The values of `df_for_ITP_info` are displayed in the output of the cell below.

In [11]:
print("The average state for each attractor in ITPs")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_for_ITP_info)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

The average state for each attractor in ITPs


is point attractor  TNFalpha  HMTs_input  \
ITP index attractor code                                                
0         ('basal', 0)                     True       0.0         0.0   
          ('transition', 0)               False       1.0         0.0   
          ('basal', 3)                    False       0.0         0.0   
1         ('basal', 1)                    False       0.0         0.0   
          ('transition', 0)               False       1.0         0.0   
          ('basal', 3)                    False       0.0         0.0   
2         ('basal', 2)                    False       0.0         0.0   
          ('transition', 0)               False       1.0         0.0   
          ('basal', 3)                    False       0.0         0.0   

                                 HMTs      NFkB       Nef  RNAs2kbC  RNAs2kbN  \
ITP index attractor code                                                        
0         ('basal', 0)       0.000000  0.000000  0.000000  0.000000  0.000000   
          ('transition', 0)  0.250000  1.000000  0.750000  0.750000  0.750000   
          ('basal', 3)       0.250000  1.000000  0.750000  0.750000  0.750000   
1         ('basal', 1)       0.142857  0.142857  0.142857  0.142857  0.142857   
          ('transition', 0)  0.250000  1.000000  0.750000  0.750000  0.750000   
          ('basal', 3)       0.250000  1.000000  0.750000  0.750000  0.750000   
2         ('basal', 2)       0.142857  0.285714  0.285714  0.285714  0.285714   
          ('transition', 0)  0.250000  1.000000  0.750000  0.750000  0.750000   
          ('basal', 3)       0.250000  1.000000  0.750000  0.750000  0.750000   

                             RNAs4kbC  RNAs4kbN  RNAs9kbC  RNAs9kbN       Rev  \
ITP index attractor code                                                        
0         ('basal', 0)            0.0  0.000000      0.00  0.000000  0.000000   
          ('transition', 0)       0.5  0.750000      0.75  0.750000  0.750000   
          ('basal', 3)            0.5  0.750000      0.75  0.750000  0.750000   
1         ('basal', 1)            0.0  0.142857      0.00  0.142857  0.142857   
          ('transition', 0)       0.5  0.750000      0.75  0.750000  0.750000   
          ('basal', 3)            0.5  0.750000      0.75  0.750000  0.750000   
2         ('basal', 2)            0.0  0.285714      0.00  0.285714  0.285714   
          ('transition', 0)       0.5  0.750000      0.75  0.750000  0.750000   
          ('basal', 3)            0.5  0.750000      0.75  0.750000  0.750000   

                                  Tat       Vpr     asRNA  p24Gag     p3LTR  \
ITP index attractor code                                                      
0         ('basal', 0)       0.000000  0.000000  0.000000    0.00  0.000000   
          ('transition', 0)  1.000000  1.000000  0.250000    0.75  0.750000   
          ('basal', 3)       1.000000  1.000000  0.250000    0.75  0.750000   
1         ('basal', 1)       0.142857  0.142857  0.142857    0.00  0.142857   
          ('transition', 0)  1.000000  1.000000  0.250000    0.75  0.750000   
          ('basal', 3)       1.000000  1.000000  0.250000    0.75  0.750000   
2         ('basal', 2)       0.285714  0.285714  0.142857    0.00  0.285714   
          ('transition', 0)  1.000000  1.000000  0.250000    0.75  0.750000   
          ('basal', 3)       1.000000  1.000000  0.250000    0.75  0.750000   

                                p5LTR    vsaRNA    vsiRNA  
ITP index attractor code                                   
0         ('basal', 0)       0.000000  0.000000  0.000000  
          ('transition', 0)  0.750000  0.750000  0.000000  
          ('basal', 3)       0.750000  0.750000  0.000000  
1         ('basal', 1)       0.142857  0.142857  0.142857  
          ('transition', 0)  0.750000  0.750000  0.000000  
          ('basal', 3)       0.750000  0.750000  0.000000  
2         ('basal', 2)       0.285714  0.285714  0.285714  
          ('transition', 0)  0.

### Selection of appropriate ITPs
You can add the `ITP index` of the ITPs you want to analyze by using the `append` method on `ITP_indices_to_analyze` in the cell directly below.<br>
Irreversibility kernel and reversion control method searches will then be performed on the selected ITPs.

The criteria for selecting ITPs are left to the user.<br>
In the analysis of established models, we used the average state information of the attractors constituting each ITP to select the ITPs corresponding to the cellular irreversible process under investigation.

If you wish to examine all constituent states of an attractor (rather than its average state), you can use:
`iATG_of_model.get_attractor_using_attr_tuple_form(attractor_code).show_states()`<br>
This command will display the constituent states of the attractor corresponding to the given attractor_code (which is a tuple type).

In [12]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

ITP_indices_to_analyze = [] # <- append the ITP indices matching the OIPC here.
ITP_indices_to_analyze.append(0)
ITP_indices_to_analyze.append(1)
ITP_indices_to_analyze.append(2)

## Analysis of selected ITPs
For each selected ITP, the irreversibility kernel is analyzed. The irreversibility kernel consists of irreversibility motifs—formed by positive feedback loops—and their corresponding coherency conditions, which stabilize each motif at the final attractor of the ITP.

Using the irreversibility kernels identified for each ITP, control strategies are subsequently explored.

### Setting feedback length for irreversibility kernel construction
In the cell below, set the value of `max_len_of_feedback_search`.<br>
This parameter determines the maximum length of positive feedback loops (in the expanded network) to be searched as candidates for the irreversibility kernel.

- If `max_len_of_feedback_search` is set to 0, the search is conducted without any length limit.

- If it is set to a positive integer, only positive feedback loops with a length less than or equal to that number will be considered.

When the network has many nodes and dense connectivity, an unrestricted search can result in excessive computational cost. Therefore, it is recommended to set this value based on the available computational resources and time constraints.

In [13]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

max_len_of_feedback_search = 0

### Analysis and visualization of the irreversibility kernel
Perform the irreversibility kernel analysis for each ITP.

In [14]:
for ITP_index in ITP_indices_to_analyze:
    ITP_object = ITP_index_map[ITP_index]
    
    # initialize the irreversibility motifs and coherency conditions in this object.
    # when you run this code more than once, you may want to clear the previous results.
    ITP_object.reset_the_info_of_irreversibility_kernel()
    
    ITP_object.find_irreversibility_kernel(max_len_of_feedback_search)

Store the results of the irreversibility kernel analysis in `df_irreversibility_kernel`.

In [15]:
ITP_indices = []
column_for_irreversibilty_motifs = []
column_for_coherency_conditions = []

for ITP_index in ITP_indices_to_analyze:
    ITP_object = ITP_index_map[ITP_index]
    for i, irreversibility_motif in enumerate(ITP_object.irreversibility_motifs):
        corresponding_coherency_condition = ITP_object.coherency_conditions[i]

        ITP_indices.append(ITP_index)
        column_for_irreversibilty_motifs.append(irreversibility_motif)
        column_for_coherency_conditions.append(corresponding_coherency_condition)

df_irreversibility_kernel = pd.DataFrame({"irreversibility motif":column_for_irreversibilty_motifs,
                                           "coherency condition":column_for_coherency_conditions},
                                          index=ITP_indices)
df_irreversibility_kernel.index.name = 'ITP index'

Structure of `df_irreversibility_kernel` is as follows.

Each row in `df_irreversibility_kernel` represents one irreversibility motif and its corresponding coherency condition for the ITP identified by the `ITP index` in that row.<br>
Since a single irreversibility kernel can consist of multiple irreversibility motifs and their corresponding coherency conditions, multiple rows can share the same ITP index.

- The `irreversibility motif` column lists the nodes that make up the irreversibility motif. In the network model’s structure, the subnetwork formed by these nodes contains combinations of positive feedbacks that constitute the irreversibility motif.
- The `coherency condition` column specifies the condition under which the `irreversibility motif` in the same row remains stable in the final attractor of the ITP. This is expressed as a dictionary where each key represents a node, and its value indicates the Boolean state that the node must maintain to satisfy the coherency condition.

In [16]:
print("Irreversibiltiy kernel for each ITP")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_irreversibility_kernel)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Irreversibiltiy kernel for each ITP


,irreversibility motif,coherency condition
ITP index,,


### This results show that it is a case where no irreversibility kernel is found